### Exercise Sheet 1

#### 1.3 Physics-informed function approximation 
Previously we used a dataset to learn solutions of the ODE:
\begin{align*}
    \partial_t^2 u(t) &= D(\partial_t u(t))^2 - g \\
    u(0) &= H \\
    \partial_t u(0) &= 0
\end{align*}
Now, we assume that the solution is not analytically known. Therefore we have to utilize the above differential equation in the training, which leads us to physics-informed neural networks.

For simplification of the implementation, we start with a fixed value for $D=0.02$.

In [1]:
import torch 

# Here all parameters are defined:
t_min, t_max = 0.0, 3.0
D = 0.02
g, H = 9.81, 50.0

# number of time points 
N_t = 50

train_iterations = 5000
learning_rate = 1.e-3

For the physics-informed training, the derivatives of the neural network have to be comupted. For this, we can use `torch.autograd.grad` (automatic differentiation). With `torch.autograd.grad` not only the gradients of neural networks can be computed but also the derivatives of general tensor operations. Here a small example for $t^2$:

In [2]:
# Create some data points
t = torch.linspace(0, 10, 11, requires_grad=True) # we need to set requires_grad=True, or else
                                                  # PyTorch will not be able to compute derivatives
u = t**2 # compute the square of the values
# Next up, we have to take the sum over all our values to compute the derivative. This has to do 
# with the implementation in PyTorch and we just have to remember to it.
# (the reason why is yet not so important)
u_sum = sum(u)
# Now we can call torch.autograd.grad:
u_t = torch.autograd.grad(u_sum, t, create_graph=True) # create_graph=True has to be set, so one can 
                                                       # later compute derivatives of higher order
# Autograd generally returns a tuple with multiple values, here we only need the first one:
print("Derivative with autograd gives:")
print(u_t[0])
print("Analytical derivative is:")
print(2*t)
print("They are in agreement!")

Derivative with autograd gives:
tensor([ 0.,  2.,  4.,  6.,  8., 10., 12., 14., 16., 18., 20.],
       grad_fn=<MulBackward0>)
Analytical derivative is:
tensor([ 0.,  2.,  4.,  6.,  8., 10., 12., 14., 16., 18., 20.],
       grad_fn=<MulBackward0>)
They are in agreement!


##### a) Working with `torch.autograd.grad`
Verify, with the help of `torch.autograd.grad`, that the previously given function
\begin{align*}
    u(t; D) &= \frac{1}{D} \left(\ln{\left( \frac{1+e^{-2\sqrt{Dg}t}}{2} \right)} - \sqrt{Dg} t \right) + H
\end{align*}
really solves the above ODE (e.g. numerically compute the derivatives and insert them into the ODE).

**Hint** : `torch.sqrt` only works for tensor-objects, you habe to either transform $D, g$ into tensors or import the `math` package for the root computation. 

In [ ]:
### TODO: verify ODE solution with torch.autograd.grad

In the following cell, the time grid, a tensor for the initial time point and the neural network are given:

In [4]:
t = torch.linspace(t_min, t_max, N_t, requires_grad=True).reshape(N_t, 1)
t_zero = torch.tensor([0.0], requires_grad=True)

model = torch.nn.Sequential(
    torch.nn.Linear(1, 20), torch.nn.Tanh(), 
    torch.nn.Linear(20, 20), torch.nn.Tanh(), 
    torch.nn.Linear(20, 1)
) 

##### b) Implementing the physics-informed Loss
Use `torch.autograd.grad` to complete the training loop, by implementing the loss for the differential equation and both initial conditions.

Each condition needs it own loss function, which are already prepared at the top of the cell.

In [ ]:
model.to("cuda")
t = t.to("cuda")
t_zero = t_zero.to("cuda")

### For the loss, we take the mean squared error and Adam for optimization.
loss_fn_ode = torch.nn.MSELoss() 
loss_fn_initial_position = torch.nn.MSELoss() 
loss_fn_initial_speed = torch.nn.MSELoss() 

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

### Training loop
for k in range(train_iterations):
    ### TODO: implement loss computation of all equations
    ### Loss for the differential equation: u_tt = D*(u_t)^2 - g
    u = ...
    u_t = ...

    loss_ode = loss_fn_ode(..., ...)

    ### Loss for initial condition: u(0) = H
    u_zero = ...
    loss_initial_position = loss_fn_initial_position(..., ...)

    ### Loss for the initial velocity: u_t(0) = 0
    
    loss_initial_speed = loss_fn_initial_speed(..., ...)

    ### Add all loss terms
    total_loss = loss_ode + loss_initial_position + loss_initial_speed

    ### Show current loss every 250 iterations:
    if k % 250 == 0 or k == train_iterations - 1:
        print("Loss at iteration %i / %i is %f" %(k, train_iterations, total_loss.item()))

    ### Optimization step
    optimizer.zero_grad()
    total_loss.backward()
    optimizer.step()

In [ ]:
import math
### Here, a check of the accruarcy of the model:
t_plot = torch.linspace(t_min, t_max, 1000).reshape(-1, 1)

### First compute error:
model.to("cpu")
model_out = model(t_plot)

sqrt_term = math.sqrt(D * g)
real_out = H - 1/D * (torch.log((1 + torch.exp(-2*sqrt_term*t_plot))/2.0) + sqrt_term*t_plot)

### Plot solution
import matplotlib.pyplot as plt
plt.figure(0, figsize=(15, 5))
plt.subplot(1, 3, 1)
plt.plot(t_plot, model_out.detach())
plt.title("Neural Network")
plt.xlabel("t")
plt.grid()
plt.subplot(1, 3, 2)
plt.plot(t_plot, real_out)
plt.title("Reference Solution")
plt.xlabel("t")
plt.grid()
plt.subplot(1, 3, 3)
plt.plot(t_plot, torch.abs(real_out - model_out).detach())
plt.title("Absolute Difference")
plt.xlabel("t")
plt.grid()
plt.tight_layout()